# Investigation of divergence between jobs market and educational programs funded from federal budget

Report from **June 21, 2019**.

## Progress

### Overall progress


- [x] Find just some list of universities
    - Obrnadzor
    - [x] Lists of discplines taught in each university
    - [x] Check if the list's outdated (DONE: it's not)
    - [x] **Number of students taught on each program?**
        - The number of students on a university-level program is **unavailable**
        - Aggregated region-wise numbers are available from Minobrnauki (see below)
    - [ ] Compare no. of organizations with some other source of data
        - [x] Compare to https://www.hse.ru/data/2017/06/29/1171183177/IO%202017.%202.%20Obrazovanie%20i%20rynok%20truda.pdf
            - [x] Filter based on bsc/msc programs and unique INNs yields results consistent with HSE report
    - [ ] Filter for PhD programs? Are there even PhD programs reported in this datasource?
- [ ] Compare the three sources suggested by Roman: Rosstat, Министерство науки и высшего образования, Social networks?
    - [x] **Rosstat** does not report educational data, it is now responsibility of Minobrnauki
    - [x] **Minobrnauki** reports data aggregated by region and we've succesfully extracted it
    - [ ] **Social networks**
- [ ] Specialization-educational program matching

### Parsing reports of Minobrnauki (VPO-1)


- [x] Initial sketch:
    - [x] `VPO1Reader`, handles a single `.xls` file.
        Possible solution:
        - [x] Table's header
            - [x] Get the number of columns.
            - [x] Find the separator between the header and the numbers.
            - [x] For each row above separator, enumerate its cells, accumulating the "atomic column"'s name.
        - [x] Data: traverse rows below separator, appending values into vectors of appropriate "atomic column"s
    - [x] `VPOSet`, handles the folder with region-wise reports.
    - [ ] Download function.
    - [x] Rewrite sheet reading subroutine as a FSM.
    - [ ] Rewrite VPO1 reader as a FSM that takes events from sheet-FSMs as input.
    - [ ] Rewrite with coroutines.
    - [ ] Use `xlrd`'s `merged_cells` instead of heuristics.
- Generator based reader:
    - [x] `read_sheet` that takes rows and produces `MetadataField`, `DFColumns` (header) and `DFDataRow` events
    - [ ] Currently, the next state is decided by each subsequent coroutine
          with `read_sheet` containing but `yield from read_region()` line.
          It is not intuitive that `read_region` also yields all the other fields on the sheet
          (by yielding control via `yield from`).
          We might want to move the responsibility of transitioning
          between "principal" states into `read_sheet`.
    - [ ] A subroutine that takes events from `read_sheet`s for each sheet in the workbook
          and produces
    - [ ] Turns out the columnames in the dataframe are to be accumulated not just
            going top-down, but also going left-to right (as for e.g. "students of the X'th course").
          Account for that.
- [ ] Feed the data into local mongodb
- [ ] Produce a `csv` with grads per region and program
          
### Matching specializations and educational programs

1. [ ] Check out [OKSO](https://classifikators.ru/okso).
1. [ ] Compare plain (e.g. wordnet) distances from "mathematics" to "IT", "javascript", etc and to, say, "machinist".
    - [ ] Construct the dictionary of `hh.ru` specializations
    - [ ] Construct the dictionary of educational programs
    - [ ] Find or train a character-based language model for Russian texts,
        trained preferrably on some tech-related corpus.
    - [ ] Check out distances between example embeddings.
    - [ ] Construct the list of specializations+programs that don't match properly based
        on just language-model.
    - [ ] Having language model embeddings, try modeling prerequisites graph. Iterate.
    - [ ] Try hyperbolic embeddings.
1. [ ] Try manually specifying some skills as dependencies for some jobs and training a net which takes two word embeddings and answers whether one is a dependency for the other.
1. [ ] This isn't going to work out of the box; research what's needed to tune it
1. [ ] Seek what datasets available that can be used for adjusting the model?
    See:
    - https://www.kaggle.com/niyamatalmass/google-job-skills
    - https://www.kaggle.com/madhab/jobposts
    - https://www.reddit.com/r/dataisbeautiful/comments/25qjpz/how_many_employees_are_moving_between_companies_oc/chjvd0g/
    - https://github.com/hhru/api
    
### Scraping social networks

The Obrnadzor data contain titles and websites of universities.
These could be used to construct a list of links to social network pages
of the universities.

- Begin with Obrnadzor data
- [ ] Construct a list of (higher edu) records that have full names and webpages
- [ ] For each: query `vk.com` for a group page (or "profile" university option)
      with such names or hyperlinks, taking only "definite" matches
- [ ] For each: construct the list of its students' profiles.
    - [ ] Construct the list of all university-page subscribers
    - [ ] Plus all `vk` users that have set this uni in their profile
    - [ ] Construct the list of groups and other uni-subscribers any of them is subscribed to
    - [ ] Use some basic community detection and somehow filter out the non-students
    - [ ] Learn to detect and filter out alumni.
    - [ ] Learn to determine current students.